In [78]:
import openai
from openai import OpenAI
import uuid
import os
import requests
import json
import urllib.parse
from datetime import datetime
from dotenv import load_dotenv
import pandas as pd
from io import StringIO

In [102]:
# authenticate

In [15]:
load_dotenv()
client = OpenAI(api_key=os.getenv('OPENAI_API_KEY'))
prolific_token = os.getenv("PROLIFIC_API_TOKEN")



In [ ]:
headers = {
    "Authorization": f"Token {prolific_token}",
    "Content-Type": "application/json"
}

In [16]:
# Fetch researcher ID
res = requests.get("https://api.prolific.com/api/v1/users/me/", headers=headers)
researcher_id = res.json()["id"]

In [17]:
researcher_id

'681781ab4a7728c1570e977c'

In [18]:
user_message = "hope u dont go bald"

In [19]:
response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "system", "content": "Given a user message, respond what do you think is the intention behind the message in a short sentence."},
        {"role": "user", "content": user_message}
    ]
)

In [20]:
intent = response.choices[0].message.content.strip()

In [21]:
intent

'The user is expressing concern or wishing that the person does not experience hair loss.'

In [24]:
# Build the full survey question
question_text = (
    f"If the user sent the message: '{user_message}'. "
    f"If the AI interpreted the intent as: "
    f"'{intent}' "
    "Do you think the AI got the intent right?"
)
print(question_text)

If the user sent the message: 'hope u dont go bald'. If the AI interpreted the intent as: 'The user is expressing concern or wishing that the person does not experience hair loss.' Do you think the AI got the intent right?


In [27]:
# Generate UUIDs for question and answers
section_id = str(uuid.uuid4())
question_id = str(uuid.uuid4())
answer_yes_id = str(uuid.uuid4())
answer_no_id = str(uuid.uuid4())

In [28]:
# Build the payload
survey_data = {
    "researcher_id": researcher_id, 
    "title": "Did the AI interpret the message correctly?",
    "sections": [
        {
            "id": section_id,
            "title": "Intent Evaluation",
            "questions": [
                {
                    "id": question_id,
                    "title": question_text,
                    "type": "single",
                    "answers": [
                        {"id": answer_yes_id, "value": "Yes"},
                        {"id": answer_no_id, "value": "No"}
                    ]
                }
            ]
        }
    ],
    "questions": [  # Flat list of questions (same structure)
        {
            "id": question_id,
            "title": question_text,
            "type": "single",
            "answers": [
                {"id": answer_yes_id, "value": "Yes"},
                {"id": answer_no_id, "value": "No"}
            ]
        }
    ]
}

In [30]:
response = requests.post(
    "https://api.prolific.com/api/v1/surveys/",
    headers=headers,  
    data=json.dumps(survey_data)
)

print(response.status_code)
print(response.json())

201
{'_id': '686593323dbca8c28ba85b5a', 'date_created': '2025-07-02T20:14:42.313221+00:00', 'date_modified': '2025-07-02T20:14:42.313227+00:00', 'researcher_id': '681781ab4a7728c1570e977c', 'sections': [{'id': '29d6e1a1-7f9f-4e73-ad69-48c0a5e44b4d', 'questions': [{'answers': [{'id': '551ded0c-a9e4-4d17-8982-75f25556b8ca', 'value': 'Yes'}, {'id': '4bd942b5-184a-41be-958c-174bc9ceffc7', 'value': 'No'}], 'id': 'e75cefd9-1251-4de9-9fbb-eee86f34ac40', 'title': "If the user sent the message: 'hope u dont go bald'. If the AI interpreted the intent as: 'The user is expressing concern or wishing that the person does not experience hair loss.' Do you think the AI got the intent right?", 'type': 'single'}], 'title': 'Intent Evaluation'}], 'questions': [{'answers': [{'id': '551ded0c-a9e4-4d17-8982-75f25556b8ca', 'value': 'Yes'}, {'id': '4bd942b5-184a-41be-958c-174bc9ceffc7', 'value': 'No'}], 'id': 'e75cefd9-1251-4de9-9fbb-eee86f34ac40', 'title': "If the user sent the message: 'hope u dont go bald'

In [31]:
survey_id = response.json()["_id"]

In [32]:
survey_id

'686593323dbca8c28ba85b5a'

In [33]:
study_config = {
    "reward": 0.50,
    "participants": 10,
    "estimated_time": 2,
    "max_time": 10,
    "age_range": "18-28",
    "countries": ["US"]
}

In [34]:
completion_code = f"INTENT_{datetime.now().strftime('%Y%m%d_%H%M%S')}"
estimated_time = int(study_config.get('estimated_time', 2))
max_time = int(study_config.get('max_time', 10))
reward_dollars = float(study_config.get('reward', 0.50))
reward_cents = int(reward_dollars * 100)
participants = int(study_config.get('participants', 10))
age_min, age_max = map(int, study_config.get('age_range', '18-25').split('-'))
countries = study_config.get('countries', ['US'])

In [35]:
prolific_country_choices = {
    'GB': '0', 'US': '1'}

In [36]:
prolific_country_ids = [prolific_country_choices[c] for c in countries if c in prolific_country_choices]

In [37]:
filters = [
    {"filter_id": "age", "selected_range": {"lower": age_min, "upper": age_max}}
]

In [38]:
if prolific_country_ids:
    filters.append({
        "filter_id": "current-country-of-residence",
        "selected_values": prolific_country_ids
    })

In [40]:
study_data = {
    "name": f"AI Intent Evaluation - {datetime.now().strftime('%Y-%m-%d %H:%M')}",
    "internal_name": f"AI Intent Study {datetime.now().strftime('%Y%m%d_%H%M%S')}",
    "description": "Evaluate whether an AI correctly identified the intent behind a user message",
    "external_study_url": f"https://prolific.com/surveys/{survey_id}",
    "completion_codes": [{
        "code": completion_code,
        "code_type": "COMPLETED",
        "actions": [{"action": "AUTOMATICALLY_APPROVE"}]
    }],
    "estimated_completion_time": estimated_time,
    "maximum_allowed_time": max_time,
    "reward": reward_cents,
    "total_available_places": participants,
    "device_compatibility": ["desktop"],
    "peripheral_requirements": [],
    "filters": filters,
    "privacy_notice": "Your data will be used for research purposes only and will be kept confidential."
}

In [41]:
study_response = requests.post(
    "https://api.prolific.com/api/v1/studies/",
    headers=headers,
    data=json.dumps(study_data)
)

In [42]:
study_response

<Response [201]>

In [43]:
study_id = study_response.json().get("id")
print(f"Study created: {study_id}")

Study created: 6865959666a4fefec3a5c60e


In [44]:
publish_response = requests.post(
        f"https://api.prolific.com/api/v1/studies/{study_id}/transition/",
        headers=headers,
        data=json.dumps({"action": "PUBLISH"})
    )

In [47]:
if publish_response.status_code == 200:
        print("Study published!")
else:
    print("Failed to publish study.")
    print("Study URL:", study_response.json().get('external_study_url'))
    print("Completion code:", completion_code)

Study published!


In [48]:
response = requests.get(
    f"https://api.prolific.com/api/v1/studies/{study_id}/",
    headers=headers
)

In [49]:
if response.status_code == 200:
    study_info = response.json()
    print("Study Status:", study_info.get("status"))
    print("Total places:", study_info.get("total_available_places"))
    print("Completed submissions:", study_info.get("total_completed"))
else:
    print("Failed to fetch study info:", response.status_code)


Study Status: ACTIVE
Total places: 10
Completed submissions: None


In [51]:
response = requests.get(
    f"https://api.prolific.com/api/v1/submissions/?study={study_id}",
    headers=headers
)

if response.status_code == 200:
    submissions = response.json().get("results", [])
    print(f"{len(submissions)} submissions found.")
    for s in submissions:
        print({
            "participant": s["participant_id"],
            "status": s["status"],  # e.g., APPROVED, RETURNED, TIMED_OUT
            "started": s["started_at"],
            "completed": s["completed_at"]
        })
else:
    print("Failed to fetch submissions:", response.status_code)


5 submissions found.
{'participant': '62851efba924ed81f6f0e7f4', 'status': 'APPROVED', 'started': '2025-07-02T20:30:33.764000Z', 'completed': '2025-07-02T20:30:42.337000Z'}
{'participant': '6713eb5e48cc2fd21195379f', 'status': 'APPROVED', 'started': '2025-07-02T20:28:50.961000Z', 'completed': '2025-07-02T20:29:10.935000Z'}
{'participant': '66a8943bc033e1485a6331a1', 'status': 'AWAITING REVIEW', 'started': '2025-07-02T20:30:59.452000Z', 'completed': '2025-07-02T20:31:16.846000Z'}
{'participant': '6113ffbf45e79bc2769db74c', 'status': 'AWAITING REVIEW', 'started': '2025-07-02T20:31:17.065000Z', 'completed': '2025-07-02T20:31:34.538000Z'}
{'participant': '67ee795beb74f4d224497dac', 'status': 'AWAITING REVIEW', 'started': '2025-07-02T20:30:59.030000Z', 'completed': '2025-07-02T20:31:33.581000Z'}


In [83]:
study_response = requests.get(
        f"https://api.prolific.com/api/v1/studies/{study_id}/",
        headers=headers)

In [97]:
study_info = study_response.json()
status = study_info.get("status")
name = study_info.get("name")
created_at = study_info.get("created_at")
total_places = study_info.get("total_available_places")
total_places_taken = study_info.get("places_taken")
created_at = datetime.fromisoformat(study_info.get("published_at").replace("Z", "+00:00")).strftime("%d %b %Y, %I:%M %p UTC")


print(f"✅ Study Name: {name}")
print(f"📊 Status: {status}")
print(f"👥 Total Places: {total_places}")
print(f"📩 Total Submissions: {total_places_taken}")
print(f"⏳ Created at: {created_at}")

✅ Study Name: AI Intent Evaluation - 2025-07-02 13:24
📊 Status: COMPLETED
👥 Total Places: 10
📩 Total Submissions: 10
⏳ Created at: 02 Jul 2025, 08:25 PM UTC


In [85]:
responses = f'https://api.prolific.com/api/v1/studies/{study_id}/export/'

In [86]:
resp = requests.get(
    responses,
    headers=headers,
)

In [88]:
df = pd.read_csv(StringIO(resp.text))

In [89]:
df

,Submission id,Participant id,Status,Custom study tncs accepted at,Started at,Completed at,Reviewed at,Archived at,Time taken,Completion code,...,Age,Sex,Ethnicity simplified,Country of birth,Country of residence,Nationality,Language,Student status,Employment status,If the user sent the message: 'hope u dont go bald'. If the AI interpreted the intent as: 'The user is expressing concern or wishing that the person does not experience hair loss.' Do you think the AI got the intent right?
0,686596802761d106f4b08c47,6713eb5e48cc2fd21195379f,APPROVED,Not Applicable,2025-07-02T20:28:50.961000Z,2025-07-02T20:29:10.935000Z,2025-07-02T20:30:54.193000Z,2025-07-02T20:29:11.376718Z,20,NOCODE,...,28,Female,White,United States,United States,United States,English,No,DATA_EXPIRED,Yes
1,686596e708451bee0f6d1780,62851efba924ed81f6f0e7f4,APPROVED,Not Applicable,2025-07-02T20:30:33.764000Z,2025-07-02T20:30:42.337000Z,2025-07-02T20:30:55.898000Z,2025-07-02T20:30:42.795459Z,9,NOCODE,...,26,Male,White,United States,United States,United States,English,No,Full-Time,Yes
2,686596fac9eb8d7f75368889,67ee795beb74f4d224497dac,APPROVED,Not Applicable,2025-07-02T20:30:59.030000Z,2025-07-02T20:31:33.581000Z,2025-07-02T20:37:46.199000Z,2025-07-02T20:31:34.057482Z,35,NOCODE,...,27,Female,Black,United States,United States,United States,English,DATA_EXPIRED,DATA_EXPIRED,No
3,68659701f8d6af19d4e3ae08,66a8943bc033e1485a6331a1,APPROVED,Not Applicable,2025-07-02T20:30:59.452000Z,2025-07-02T20:31:16.846000Z,2025-07-02T20:37:48.161000Z,2025-07-02T20:31:17.281863Z,18,NOCODE,...,25,Female,White,Russian Federation,United States,United States,Russian,DATA_EXPIRED,DATA_EXPIRED,Yes
4,6865971274316bb412d69b72,6113ffbf45e79bc2769db74c,APPROVED,Not Applicable,2025-07-02T20:31:17.065000Z,2025-07-02T20:31:34.538000Z,2025-07-02T20:37:51.690000Z,2025-07-02T20:31:35.006916Z,18,NOCODE,...,25,Female,Black,Nigeria,United States,United States,English,Yes,Full-Time,Yes
5,6865977b32cb60e55bbfe44a,655f9bb3070d42d4bc67e866,APPROVED,Not Applicable,2025-07-02T20:33:02.383000Z,2025-07-02T20:33:17.458000Z,2025-07-02T20:37:50.742000Z,2025-07-02T20:33:17.908606Z,16,NOCODE,...,26,Female,Black,United States,United States,United States,English,No,Full-Time,No
6,686597840343731b34c7f8ea,65a4333efc75f965e7fc0cb5,APPROVED,Not Applicable,2025-07-02T20:33:10.142000Z,2025-07-02T20:33:27.311000Z,2025-07-02T20:37:54.487000Z,2025-07-02T20:33:27.718608Z,18,NOCODE,...,19,Male,Asian,United States,United States,United States,English,Yes,Other,Yes
7,686597f4931f72878a39a29f,61647de75314468d5cb57f93,APPROVED,Not Applicable,2025-07-02T20:35:04.091000Z,2025-07-02T20:35:56.019000Z,2025-07-02T20:37:52.761000Z,2025-07-02T20:35:56.447923Z,52,NOCODE,...,26,Male,White,United States,United States,United States,English,No,Unemployed (and job seeking),Yes
8,686597fa6821ab422aad640f,66465b2522d14ab162ff2dbd,APPROVED,Not Applicable,2025-07-02T20:35:08.944000Z,2025-07-02T20:35:27.903000Z,2025-07-02T20:37:53.612000Z,2025-07-02T20:35:28.318144Z,19,NOCODE,...,21,Female,White,United States,United States,United States,English,DATA_EXPIRED,Part-Time,No
9,68659b549bce3a7ab6b0913e,667d08e12ac1bcbdd829a936,APPROVED,Not Applicable,2025-07-02T20:49:26.661000Z,2025-07-02T20:49:37.590000Z,2025-07-02T20:51:11.322000Z,2025-07-02T20:49:38.141414Z,11,NOCODE,...,21,Female,Asian,United States,United States,United States,English,DATA_EXPIRED,DATA_EXPIRED,Yes


In [99]:
completed_times = pd.to_datetime(df["Completed at"], errors="coerce")
latest_completion = completed_times.dropna().max()
formatted_time = latest_completion.strftime("%d %b %Y, %I:%M %p UTC")

In [100]:
formatted_time

'02 Jul 2025, 08:49 PM UTC'

In [ ]:
totalstudytime = formatted_time - completed_times